In [8]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
from tqdm import tqdm

IMG_PX_SIZE = 80

data_dir = 'C:/users/setcodestofire/documents/mygithub/brain/Image'
dataset  = os.listdir(data_dir)

In [2]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

def mean(l):
    return sum(l) / len(l)

def process_data(data):
    
    path = data_dir +'/'+ str(data)
    img3d = nib.load(path)
    a = np.array(img3d.get_data())
    
    slice_img=[]
    if(img3d.header['dim'][4]==1):
        for i in range(a.shape[2]):
            slice_img.append(cv2.resize(np.array(a[:,:,i]),(IMG_PX_SIZE,IMG_PX_SIZE)))
    else:
        for i in range(a.shape[2]):
            slice_img.append(cv2.resize(np.array(a[:,:,i,1]),(IMG_PX_SIZE,IMG_PX_SIZE)))
    
    
    new_slices = []
    
    chunk_sizes = math.ceil(len(slice_img) / 16)
    for slice_chunk in chunks(slice_img, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)
    
    return np.array(new_slices)

img_data = process_data(dataset[0])

In [3]:
import tflearn
LR = 0.001

from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None,IMG_PX_SIZE,IMG_PX_SIZE,1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
#convnet = regression(convnet, optimizer='adam', learning_rate=LR,loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet)

model.load('model.tflearn')
model.predict(img_data[8].reshape(-1, 80,80, 1))

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
INFO:tensorflow:Restoring parameters from C:\Users\setCodesToFire\Documents\MyGithub\Brain\model.tflearn


array([[ 1.,  0.]], dtype=float32)